In [43]:
load('../../../data/impactAnnotated.RData')

In [44]:
impact.ann.df$class_label <- impact.ann.df$confidence_class %in% c('MANUAL_OK', 'AUTO_OK')

In [45]:
impact.ann.subset.df <- subset(impact.ann.df, confidence_class %in% c('UNLIKELY', 'MANUAL_OK', 'AUTO_OK'))

In [46]:
labelTrain.subset <- impact.ann.subset.df$class_label 

In [47]:
table(impact.ann.subset.df$confidence_class)


  AUTO_OK MANUAL_OK  UNLIKELY 
   191228      7113     15317 

In [48]:
#colnames(impact.ann.subset.df)

In [49]:
selFeatures <- c('Consequence',
                 'cgType',
                #'t_depth',
                 't_vaf',
                 't_alt_count',
                 'n_depth',
                 'n_vaf',
                 'n_alt_count',
                 't_ref_plus_count',
                 't_ref_neg_count',
                 'occurence_in_normals',
                 #'minor_contamination',
                 'SIFT_score',
                 #'Polyphen2_HDIV_score',
                 #'LRT_score',
                 #'MutationTaster_score',
                 #'MutationAssessor_score',
                 #'FATHMM_score',
                 #'PROVEAN_score',
                 #'VEST3_score',
                 #'CADD_raw',
                 #'DANN_score',
                 #'fathmm.MKL_coding_score',
                 #'MetaSVM_score',
                 #'MetaLR_score',
                 #'integrated_fitCons_score',
                 #'X1000g2015aug_all',
                 'Kaviar_AF',
                 'isOncogene',
                 'Total.onco',
                 'cosmicCount'
                )
variantTrain <- impact.ann.subset.df [, selFeatures]
nrow(variantTrain)

[1] 213658

In [50]:
variantTrain.corr <- variantTrain
variantTrain.corr$Consequence <- as.factor(variantTrain.corr$Consequence)
variantTrain.corr$t_vaf <- as.numeric(variantTrain.corr$t_vaf)
variantTrain.corr$t_alt_count <- as.numeric(variantTrain.corr$t_alt_count)
variantTrain.corr$n_depth <- as.numeric(variantTrain.corr$n_depth)
variantTrain.corr$t_ref_plus_ratio<- as.numeric(variantTrain.corr$t_ref_plus_count)  / 
(as.numeric(variantTrain.corr$t_ref_plus_count) + as.numeric(variantTrain.corr$t_ref_neg_count))

variantTrain.corr$SIFT_score<- as.numeric(variantTrain.corr$SIFT_score)
variantTrain.corr$Kaviar_AF<- as.numeric(variantTrain.corr$Kaviar_AF)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [51]:
variantTrain.corr$occurence_in_normals <- as.numeric(as.character(gsub(';.*', '', variantTrain.corr$occurence_in_normals)))

In [52]:
t <- as.numeric(as.character(gsub(';.*', '', variantTrain.corr$occurence_in_normals)))
t[is.na(t)] <- 0
variantTrain.corr$occurence_in_normals <- t

In [53]:
variantTrain.corr$SIFT_score[is.na(variantTrain.corr$SIFT_score)] <- 1

In [54]:
variantTrain.corr$t_ref_plus_ratio[is.na(variantTrain.corr$t_ref_plus_ratio)] <- 0.5

In [55]:
variantTrain.corr$Kaviar_AF[is.na(variantTrain.corr$Kaviar_AF)] <- 0

In [56]:
table(variantTrain.corr$Total.onco)


    Inconclusive   Likely Neutral Likely Oncogenic        Oncogenic 
             466              248            42378            14558 
            null      NotInOncoKb 
               0           156008 

In [57]:
table(variantTrain.corr$isOncogene)
levels(variantTrain.corr$isOncogene) <- c(levels(variantTrain.corr$isOncogene), 'unclear')
variantTrain.corr$isOncogene[is.na(variantTrain.corr$isOncogene)] <- 'unclear'


          Yes 
149137  58588 

In [58]:
table(variantTrain.corr$cgType)
sum(is.na(variantTrain.corr))


 Oncogene Recessive   Unknown 
    58588    149137      5933 

[1] 0

In [59]:
save(variantTrain.corr, labelTrain.subset, file='../../../data/trainingData.RData')

In [60]:
table(labelTrain.subset)

labelTrain.subset
 FALSE   TRUE 
 15317 198341 

In [63]:
toSubsample <- !labelTrain.subset

In [66]:
realMutationsSelected <- sample(which(labelTrain.subset==TRUE), sum(labelTrain.subset==FALSE))

In [68]:
toSubsample[realMutationsSelected] <- TRUE

In [69]:
sum(toSubsample)

[1] 30634

In [70]:
table(labelTrain.subset[toSubsample])


FALSE  TRUE 
15317 15317 

In [71]:
variantTrain.corr <- variantTrain.corr[toSubsample,]
labelTrain.subset <- labelTrain.subset[toSubsample]

In [72]:
save(variantTrain.corr, labelTrain.subset, file='../../../data/trainingData.Balanced.RData')